## pommerman

ボンバーマンの学習を行う

![pommerman](https://www.pommerman.com/static/media/pommerman.abbcd943.gif)

前回コンペの対戦などが見れる

https://www.pommerman.com/leaderboard

## 今年のルール

free-for-all, teamに加えてエージェント間の通信が可能な `PommeRadioCompetition-v2` が追加された

https://www.pommerman.com/competitions

## 今までの環境との違い

- 状態が爆発的に多い
- 対環境で自分の行動だけが環境に影響があって予測しやすいが、pommermanはチームや相手が複数いる。
- 相手も増えて難しくなるが、仲間と連携も可能だったり奥深さも増す

## 用意されたエージェント

- [RandomAgent](https://github.com/MultiAgentLearning/playground/blob/master/pommerman/agents/random_agent.py) ランダムに動くエージェント (すぐ自爆するので弱い)
- [SimpleAgent](https://github.com/MultiAgentLearning/playground/blob/master/pommerman/agents/simple_agent.py) 人間が考えたロジックで動作するエージェント(かなり強い)
- [PlayerAgent](https://github.com/MultiAgentLearning/playground/blob/master/pommerman/agents/player_agent.py) 人間が操作可能なエージェント。キーボード上下左右とスペースで操作可能
- 以下DockerAgentとHttpAgentを利用してサーバーを立てて対戦が可能
  - [HttpAgent](https://github.com/MultiAgentLearning/playground/blob/master/pommerman/agents/http_agent.py)
  - DockerAgent [Dockerfile](https://github.com/MultiAgentLearning/playground/blob/master/examples/docker-agent/Dockerfile) [simple_ffa_run.py](https://github.com/MultiAgentLearning/playground/blob/master/examples/simple_ffa_run.py#L20)

## エージェントの自作

シンプルな停止エージェント。

```python
from pommerman.agents import BaseAgent

class StoppingAgent(BaseAgent):
    def act(self, obs, action_space):
        # 0 stop, 1 up, 2 down, 3 left, 4 right, 5 bom
        return 0
```

obsにはボードの状態やアイテム取得状況など必要な情報が全て入っている。

アクションの種類やボードの数値については以下ドキュメントに書かれている。
https://github.com/MultiAgentLearning/playground/tree/master/pommerman

## 各エージェントの動作とシンプルな実行方法

`pommerman_simple_agents.py` に実装してます。

jupyter notebook上でrenderする方法が探せなかったのでlocalで実行します。

```python
env = pommerman.make('PommeFFACompetition-v0', [
    agents.PlayerAgent(),
    agents.SimpleAgent(),
    agents.RandomAgent(),
    StoppingAgent(),
])

for i_episode in range(1):
    state = env.reset()
    done = False
    while not done:
        env.render()
        actions = env.act(state)
        state, reward, done, info = env.step(actions)
        if done:
            win_player = info['winners'][0] + 1
            print(f'win {win_player}P')
    print(f'Episode {i_episode} finished')
env.close()
```

## 強化学習で実装するエージェント

重要な要素

- `Rewards.get_rewards` のリワード設計
- `EnvWrapper.featurize` の学習すべき状態設計
- `create_model` モデル設計
- `create_dqn` DQNパラメータ調整

In [1]:
%matplotlib inline

import os
import io
import base64

from IPython import display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import gym

from pommerman.agents import BaseAgent, SimpleAgent
from pommerman.configs import ffa_v0_fast_env
from pommerman.envs.v0 import Pomme

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from keras.optimizers import Adam

from rl.core import Env, Processor
from rl.agents.dqn import DQNAgent
from rl.policy import MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import Callback

Using TensorFlow backend.


In [2]:
class Rewards:
    def __init__(self):
        pass

    def get_rewards(self, obs_now, action_now, reward):

        # TODO: リワードの調整

        # ボードの詳細
        # https://github.com/MultiAgentLearning/playground/tree/master/pommerman

        # 例: 爆弾獲得
        if self.ammo < obs_now['ammo']:
            reward += 0.2
        self.ammo = obs_now['ammo']

        # 例: 火力獲得
        if self.blast_strength < obs_now['blast_strength']:
            reward += 0.2
        self.blast_strength = obs_now['blast_strength']

        # 例: キック獲得
        if self.can_kick is False and obs_now['can_kick'] is True:
            reward += 0.2
        self.can_kick = obs_now['can_kick']

        # 例: 生存step数ペナルティ
        reward += obs_now['step_count'] * -0.0001

        # print(reward)

        return reward

    def reset(self):
        self.ammo = 1
        self.blast_strength = 2
        self.can_kick = False

        # print(f'Rewards.reset')

In [3]:
class EnvWrapper(Env):
    def __init__(self, gym):
        self.gym = gym
        self.rewardShaping = Rewards()

    def __del__(self):
        self.close()

    def __str__(self):
        return '<{} instance>'.format(type(self).__name__)

    def render(self, mode='human', close=False):
        self.gym.render(mode=mode, close=close)

    def close(self):
        self.gym.close()

    def seed(self, seed=None):
        raise self.gym.seed(seed)

    def configure(self, *args, **kwargs):
        raise NotImplementedError()

    def reset(self):
        # print('EnvWrapper.reset')
        self.rewardShaping.reset()
        obs = self.gym.reset()
        agent_obs = self.featurize(obs[self.gym.training_agent])
        return agent_obs

    def step(self, action):
        # print(f'EnvWrapper.step action = {action}')
        obs = self.gym.get_observations()
        all_actions = self.gym.act(obs)
        all_actions.insert(self.gym.training_agent, action)
        state, reward, terminal, info = self.gym.step(all_actions)
        action = all_actions[self.gym.training_agent]
        agent_state = self.featurize(state[self.gym.training_agent])
        agent_reward = reward[self.gym.training_agent]

        agent_reward = self.rewardShaping.get_rewards(
            obs[self.gym.training_agent],
            action,
            agent_reward,
        )

        return agent_state, agent_reward, terminal, info

    def featurize(self, obs):

        # TODO: トレーニングエージェントのobsを加工して返す

        # 例: 自分は10、敵は11に変更 (11, 11, 12)
        # board = obs['board']
        # board = np.where(13 == board, 10, board)
        # board = np.where(10 < board, 11, board)

        # 例: カテゴリカルデータに変更
        # from tensorflow.keras.utils import to_categorical
        # nb_classes = 12
        # board = to_categorical(board, nb_classes).astype(np.float32)
        # print(board.shape)

        return obs['board']

In [4]:
class CustomProcessor(Processor):
    def process_state_batch(self, batch):
        # print(f'CustomProcessor.process_state_batch batch = {batch.shape}')
        return batch

    def process_info(self, info):
        info['result'] = info['result'].value
        return info

In [5]:
class DQN(BaseAgent):
    def act(self, obs, action_space):
        pass

In [6]:
def get_env():
    config = ffa_v0_fast_env()
    env = Pomme(**config["env_kwargs"])
    # env.seed(0)

    agent_id = 0

    agents = [
        DQN(config["agent"](0, config["game_type"])),
        SimpleAgent(config["agent"](1, config["game_type"])),
        SimpleAgent(config["agent"](2, config["game_type"])),
        SimpleAgent(config["agent"](3, config["game_type"])),
    ]

    env.set_agents(agents)

    env.set_training_agent(agents[agent_id].agent_id)
    env.set_init_game_state(None)

    return env

In [7]:
def create_model(input_shape, output_units, history_length):
    model = Sequential()

    model.add(Conv2D(
        32,
        kernel_size=2,
        strides=(1, 1),
        input_shape=(history_length, input_shape[0], input_shape[1]),
        activation='relu',
    ))
    model.add(Conv2D(
        64,
        kernel_size=2,
        strides=(1, 1),
        activation='relu'),
    )
    model.add(Conv2D(
        64,
        kernel_size=2,
        strides=(1, 1),
        activation='relu',
    ))
    model.add(Flatten())
    model.add(Dense(
        units=128,
        activation='relu',
    ))
    model.add(Dense(
        units=128,
        activation='relu',
    ))
    model.add(Dense(
        units=output_units,
        activation='linear',
    ))

    # print(model.input_shape)
    model.summary()

    return model

In [8]:
def create_dqn(model, history_length):
    memory = SequentialMemory(limit=500000, window_length=history_length)
    policy = MaxBoltzmannQPolicy()

    dqn = DQNAgent(
        model=model,
        nb_actions=model.output_shape[1],
        memory=memory,
        policy=policy,
        processor=CustomProcessor(),
        nb_steps_warmup=512,
        enable_dueling_network=True,
        dueling_type='avg',
        target_model_update=5e2,
        batch_size=32,
    )
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])

    return dqn

In [9]:
weight_path = 'models/pommerman/keras_weights.h5'
history_length = 4
input_shape = (11, 11)
output_units = 6

In [10]:
env = get_env()

In [11]:
env_wrapper = EnvWrapper(env)

In [12]:
model = create_model(input_shape, output_units, history_length)
dqn = create_dqn(model, history_length)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 10, 32)         1440      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 9, 64)          8256      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 8, 64)          16448     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [13]:
if os.path.exists(weight_path):
    dqn.load_weights(weight_path)

In [14]:
try:
    dqn.fit(
        env_wrapper,
        nb_steps=2000000,  # 8h
        visualize=True,
        nb_max_episode_steps=env._max_steps,
    )
except KeyboardInterrupt:
    pass
finally:
    dqn.save_weights(weight_path, overwrite=True)

Training for 2000000 steps ...
Interval 1 (0 steps performed)

10000/10000 [==============================] - 171s 17ms/step - reward: -0.0372
400 episodes - episode_reward: -0.931 [-1.365, -0.315] - loss: 0.014 - mae: 0.404 - mean_q: 0.524 - result: 2.949

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 205s 21ms/step - reward: -0.0303
329 episodes - episode_reward: -0.920 [-1.263, -0.108] - loss: 0.011 - mae: 0.216 - mean_q: 0.235 - result: 2.960

Interval 3 (20000 steps performed)
 2831/10000 [=======>......................] - ETA: 2:31 - reward: -0.0336done, took 436.535 seconds


## 結果

CPUで数時間程度では全然学習できない。

爆弾を置くと自分で自爆するので置かない方が安全だと気づいてしまう。

動かないplayerと対戦させたり、壁壊すことに報酬与えたり、爆弾置けるのに置かないと罰を与えたりしたが若干爆風を避けたかと思う程度で全然SimpleAgentに勝てるレベルでは学習できなかった。

## 考察

1. トレーニングエージェントのobsを加工して返す `EnvWrapper.featurize`
  - obsにはボードの状態やアイテムの取得状況など必要な状態は全て揃っているので、そのデータを加工してネットワークのinput_shapeを変更する
  - 全部の情報を利用すると情報過剰で収束にかなり時間がかかる。最初は周りの9マスのみに変更するなど情報量を削って試してみる
  - 1と2は連続値ではなくidなのでカテゴリカルデータに変更する
    - [[1, 2], [3, 4]] => [[[1,0,0,0]], [0,1,0,0]], [[0,0,1,0], [0,0,0,1]]]
  - 自分(10)、敵(11,12,13)なので自分以外は敵としてデータを調整するなど
2. リワードの調整を行う `Rewards.get_rewards`
  - 最初に敵倒せるなど不可能なのでよく起こりうることに対して報酬を与えれないか考える
    - 壁壊したら報酬与える
    - アイテムとったら報酬与える
  - ボム置かなくなる対策
    - ボムを置いたら報酬を与える
    - 時間経過(step)ごとにマイナス報酬を増大/減少させる
      - 生存のために減少させると動かなくなりうる
      - 挑戦させるためにマイナス報酬を増大させる
3. パラメータ調整やネットワークを変えてみる
  - DQNのパラメータ、policyや割引率など変えてみる
  - DQN以外にもNAFAgent,DDPGAgent,SARSAAgent,CEMAgent,PPOAgentなどいっぱいある
  - 教師あり学習でも良いしニューラルネットワークでなくても良い。別フレームワークでも良い
4. その他のアイディアを考える
  - SimpleAgentには全く勝てないのでStoppingAgentを実装して動かない相手と対戦して知識をつける
  - BaseLineAgentの動作を教師あり学習させたあと、蒸留してネットワークを縮める
  - ActionFiterのような知識(半ロジック)を埋め込む
  - PlayerAgent(キーボードで人間が操作可能)なエージェントのログを教師あり学習で学習する
  - 学習にかなり時間かかるのでtry and error難しい。GPUなど高火力で攻める

## コンペランキングと実装から考察

https://www.pommerman.com/leaderboard
https://challonge.com/ja/runningPommermanNeurips

- ffa 1位 [YichenGong/Agent47Agent](https://github.com/YichenGong/Agent47Agent)
- team 1位, 2~3位
  - [記事](https://www.ibm.com/blogs/think/jp-ja/eal-time-sequential-decision-making/)
  - [スライド](https://www.slideshare.net/TakayukiOsogami/pommerman?next_slideshow=1)
  - [Docker hub](https://hub.docker.com/layers/multiagentlearning/hakozakijunctions/latest/images/sha256-1ef23f0e35a6a404e7b4cb7a4356ff8d10748f1fd7f8d9fef2e76fb4eda8ce41)
  - [実装](https://github.com/takahasixxx/GGG)
  - [実装 アクション決定ロジック](https://github.com/takahasixxx/GGG/blob/c88fac39964ce74ff0084d37a7b00937be6088e9/src/com/ibm/trl/BBM/mains/ActionEvaluator.java#L42)
- 学習エージェント2位 skynet
  - [記事](https://www.borealisai.com/en/blog/pommerman-team-competition-or-how-we-learned-stop-worrying-and-love-battle/)
  - [ActionFilterの実装](https://github.com/BorealisAI/pommerman-baseline)
  - [資料](https://www.researchgate.net/publication/332897569_Skynet_A_Top_Deep_RL_Agent_in_the_Inaugural_Pommerman_Team_Competition)
  - [論文](https://arxiv.org/abs/1907.11788) [論文](https://arxiv.org/abs/1905.01360)
  - Borealis AI team 学習エージェントカテゴリで2位 (非学習)ヒューリスティックエージェントを含むグローバルランキングで5位になりました。
  - 3番目のブロックであるActionFilterモジュールは、エージェントに事前知識をインストールするという哲学に基づいて、エージェントにすべきでないことを伝えることで構築され、エージェントが試行錯誤、つまり学習によって何をすべきかを発見できるようにしました。



状態が多すぎるゲームにおいては完全に深層学習だけでは(現実的ではない学習量になって)実装できず、ある程度ロジックによる補助が必要そう。

## 参考

keras-rl実装
- https://github.com/kenkangg/Multi-Agent-Cooperative-Competitive-Environments
- https://github.com/Borzyszkowski/RL-Bomberman-Gradient

tensorforce実装
- https://github.com/MultiAgentLearning/playground/blob/master/notebooks/Playground.ipynb

各学習結果
- https://github.com/papkov/pommerman-x/blob/master/DQN.ipynb
- https://github.com/tambetm/pommerman-baselines/tree/master/imitation